이 자료는 Hugging Face Pipeline API를 이용한 챗봇 튜토리얼로 작성되었습니다.  
2025년 9월 14일에 정상 동작을 확인하였습니다.  

1. Hugging Face 회원가입(Sign Up)

    - [Hugging Face 바로가기: https://huggingface.co/](https://huggingface.co/)  
    - Sign Up 따라하기  

2. Hugging Face의 Access Token 생성  

    - 우측상단 (三) 메뉴 -> Settings -> Tokens -> +Create new token
    - 읽기 전용(Read) / 쓰기(Write) / 관리자(Admin) 중 목적에 맞게 발급  
    - 발급된 코드는 다시 볼 수 없기 때문에 별도로 보관  

In [ ]:
from huggingface_hub import login
# login(token="hf_xxx_your_token")

3. Transformer 설치  

In [ ]:
!pip install --upgrade transformers

4. Colab에서 Hugging Face의 Access Token을 이용하여 로그인(LogIn)  

In [ ]:
!hf auth login --token token --add-to-git-credential

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/hf_api.py", line 1782, in whoami
    hf_raise_for_status(r)
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 482, in hf_raise_for_status
    raise _format(HfHubHTTPError, str(e), response) from e
huggingface_hub.errors.HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2 (Request ID: Root=1-68c6a538-44c846d85452

5. Git credential 설정 하기  

In [ ]:
!git config --global credential.helper store

6. Hugging Face 로그인 상태 확인  

In [ ]:
!hf auth whoami

gislee


7. 질의응답(Q&A) 모델 불러오기 (bert-large-uncased-whole-word-masking-finetuned-squad)

In [ ]:
from transformers import pipeline

qa = pipeline("question-answering")

context = """AI agents are intelligent systems that can perceive the environment,
make decisions, and take actions autonomously. They are increasingly powered by
large language models (LLMs) such as GPT or LLaMA."""

question = "What powers modern AI agents?"
result = qa(question=question, context=context)
print("질문:", question)
print("답변:", result['answer'])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cuda:0


질문: What powers modern AI agents?
답변: large language models


8. Hugging Face Pipeline API를 이용한 챗봇

In [1]:
# 1) 설치
!pip -q install "transformers>=4.44.0" "accelerate>=0.33.0" bitsandbytes "sentencepiece>=0.1.99" gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 9.0 MB/s eta 0:00:00


In [2]:
# 2) 환경 점검 & 모델 선택 (GPU 유무에 따라 자동)
import torch, subprocess, re, os, sys

def has_cuda():
    return torch.cuda.is_available()

def gpu_name():
    if not has_cuda(): return "CPU"
    try:
        out = subprocess.check_output(["nvidia-smi", "-L"]).decode()
        return out.strip().split("\n")[0]
    except Exception:
        return "CUDA"

gpu = gpu_name()
print("🔧 Device:", "CUDA" if has_cuda() else "CPU", "|", gpu)

# VRAM 추정
vram_gb = 0
if has_cuda():
    props = torch.cuda.get_device_properties(0)
    vram_gb = round(props.total_memory / 1024**3, 1)
print("🧠 VRAM(GB):", vram_gb)

# 모델 자동 선택
# - GPU 12GB 이상: 7B 권장 (4bit 로딩)
# - 그 외: 1.5B (CPU/저GPU도 동작)
if has_cuda() and vram_gb >= 12:
    MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
    LOAD_4BIT = True
else:
    MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
    LOAD_4BIT = False  # CPU에서는 4bit 로딩 불가

print(f"✅ Selected model: {MODEL_NAME} | 4bit={LOAD_4BIT}")

🔧 Device: CUDA | GPU 0: Tesla T4 (UUID: GPU-52ee9fb0-7b81-4201-1207-2fef718658e7)
🧠 VRAM(GB): 14.7
✅ Selected model: Qwen/Qwen2.5-7B-Instruct | 4bit=True


In [3]:
# 3) 모델 & 토크나이저 로드 (지시문 튜닝 모델용 chat_template 사용)
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers

dtype = torch.bfloat16 if has_cuda() else torch.float32

kwargs = {
    "torch_dtype": dtype,
    "low_cpu_mem_usage": True,
}

if has_cuda() and LOAD_4BIT:
    kwargs.update(dict(
        device_map="auto",
        load_in_4bit=True
    ))
elif has_cuda():
    kwargs.update(dict(device_map="auto"))

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True, **kwargs)

print("✅ Model ready.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Model ready.


In [4]:
# 4) 생성 함수 (대화 이력 + 시스템 프롬프트 + 안전 파라미터)
import random
from typing import List, Dict

SYSTEM_PROMPT = (
    "You are a helpful, concise assistant for a university AI Agent course. "
    "Answer in the user's language (Korean or English). "
    "Provide accurate, step-by-step, and safe information."
)

GEN_CFG = dict(
    max_new_tokens=512,
    temperature=0.7,        # 안정+창의 균형
    top_p=0.9,
    repetition_penalty=1.05,
    do_sample=True
)

def generate_reply(history: List[Dict[str, str]], user_msg: str) -> str:
    # history: [{"role":"user"/"assistant", "content":"..."}]
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    messages += history
    messages.append({"role": "user", "content": user_msg})

    # chat template 적용 (지시문 튜닝 모델 필수)
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    if has_cuda():
        input_ids = input_ids.to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            eos_token_id=tokenizer.eos_token_id,
            **GEN_CFG
        )

    out_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 마지막 assistant 응답만 추출 (chat_template 기준으로 단순 파싱)
    # chat 템플릿 포맷에 의존하지 않는 안전한 방법: 마지막 messages 길이 기준으로 잘라내기
    # 간단하게는 사용자 질문 이후 텍스트를 가져오는 휴리스틱 사용:
    if "assistant" in out_text.lower():
        # 템플릿에 따라 "assistant" 라벨이 들어가는 경우가 있어 마지막 부분을 슬라이스
        split_tok = "assistant"
        idx = out_text.lower().rfind(split_tok)
        candidate = out_text[idx+len(split_tok):].strip(": \n")
        # 과도한 프리앰블 제거
        return candidate.strip()
    else:
        # 템플릿이 깔끔히 없으면, 입력 프롬프트 길이만큼 자르고 뒤를 답변으로 사용
        prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        reply = out_text[len(prompt_text):].strip()
        return reply

# 테스트
print("🧪 테스트:", generate_reply([], "간단히 자기소개해줘"))

🧪 테스트: 안녕하세요! 저는 AI 에이전트 코스의 학생입니다. 다양한 기술과 문제 해결 능력을 배우기 위해 이 과정을 수강하고 있습니다. 질문에 답변하거나 도와드릴 수 있도록 항상 준비되어 있습니다. 무엇을 도와드릴까요?


In [5]:
# 5) Gradio 채팅 UI (대화 기록 유지 / 리셋)
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 🤖 고품질 지시문 튜닝 챗봇 (HF Transformers)")
    gr.Markdown(f"- 모델: **{MODEL_NAME}**  |  언어: 한국어/영어 자동 대응  \n- 강의/학습용: 정확하고 간결한 답변 지향")
    chat = gr.Chatbot(height=420)
    msg = gr.Textbox(placeholder="질문을 입력하세요 (예: 코사인 유사도를 설명해줘).")
    clear = gr.Button("대화 리셋")

    state = gr.State([])  # [{"role":..., "content":...}, ...]

    def respond(user_text, history_state):
        history_state = history_state or []
        bot_text = generate_reply(history_state, user_text)
        history_state = history_state + [{"role":"user","content":user_text},
                                         {"role":"assistant","content":bot_text}]
        visible_chat = [(history_state[i]["content"], history_state[i+1]["content"])
                        for i in range(0, len(history_state), 2)]
        return "", visible_chat, history_state

    def reset():
        return [], []

    msg.submit(respond, [msg, state], [msg, chat, state])
    clear.click(fn=reset, inputs=None, outputs=[chat, state])

demo.launch(debug=False)


/tmp/ipython-input-3975524711.py:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot(height=420)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://669e1bb376134397be.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
